<a href="https://colab.research.google.com/github/robert2018251032/Image-Directory-to-CSV/blob/master/Tomato_Dataset_Predictions_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

Reading Dataset

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '../input/tomato/New Plant Diseases Dataset(Augmented)/train'
valid_path = '../input/tomato/New Plant Diseases Dataset(Augmented)/valid'

VGG19 Model

In [ ]:
vgg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in vgg19.layers:
    layer.trainable = False

In [ ]:
folders = glob('../input/tomato/New Plant Diseases Dataset(Augmented)/train/*')

In [ ]:
folders

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg19.output)

In [ ]:
len(folders)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg19.input, outputs=prediction)

Model Summary

In [ ]:
model.summary()

In [ ]:
optimizer = RMSprop(0.001) #learning rate = 0.001

model.compile(
  loss = 'categorical_crossentropy',
  optimizer = optimizer,
  metrics = ['accuracy']
)

Create data generator for training and testing set. Augmentation process is only done for training dataset.

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('../input/tomato/New Plant Diseases Dataset(Augmented)/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('../input/tomato/New Plant Diseases Dataset(Augmented)/valid',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Fitting the Model

In [ ]:
# fit the model
es = EarlyStopping(patience=20, restore_best_weights=True)

r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=es
)

Validation Accuracy

In [ ]:
valid_loss, valid_acc = model.evaluate_generator(test_set, steps=len(test_set))
print(f"Final validation accuracy: {valid_acc*100:.2f}%")

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('Acc-Val_acc')

In [ ]:
model.save('model_vgg19.h5')
print("Model saved successfully...")

Prediction

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred